In [3]:
import numpy as np 
import pandas as pd
#import calendar
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [4]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))
    
#Uno los dataFrame
metadata = pd.DataFrame()
for dataframe in dataframes:
    metadata = pd.concat([metadata, dataframe]).drop_duplicates().reset_index(drop=True)

In [5]:
data = metadata.loc[(metadata.state_name.str.contains('Buenos Aires')) | (metadata.state_name.str.contains\
                ('Capital Federal'))  | (metadata.state_name.str.contains\
                ('Bs.As') ),:]
data.shape
# 1.283.741 entries antes, ahora 242290

(242290, 27)

In [6]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test_visualiz = test.loc[:,['state_name', 'id']].groupby('state_name').agg(np.size)
test_visualiz.head(10)

,id
state_name,
Bs.As. G.B.A. Zona Norte,2
Bs.As. G.B.A. Zona Oeste,3101
Bs.As. G.B.A. Zona Sur,2215
Buenos Aires Interior,1
Capital Federal,8847


In [7]:
data['state_name'].tail()

296472                 Capital Federal
296473        Bs.As. G.B.A. Zona Norte
296474                 Capital Federal
296475    Buenos Aires Costa Atlántica
296476                 Capital Federal
Name: state_name, dtype: object

In [8]:
# Depuración del set de test
#Completo los valores NaN con promedios para realizar la predicción
prom_surface_total_in_m2 = test['surface_total_in_m2'].mean()
prom_surface_covered_in_m2 = test['surface_covered_in_m2'].mean()
prom_rooms = int(test['rooms'].mean())
prom_lat = test['lat'].mean()
prom_lon = test['lon'].mean()

test_float = test
test_float['surface_total_in_m2'].fillna(prom_surface_total_in_m2, inplace=True)
test_float['surface_covered_in_m2'].fillna(prom_surface_covered_in_m2, inplace=True)
test_float['rooms'].fillna(prom_rooms, inplace=True)
test_float['lat'].fillna(prom_lat, inplace=True)
test_float['lon'].fillna(prom_lon, inplace=True)


test_float.head(100)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.000000,359.471588,NaN,2.0,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,-58.465820,0.000000,359.471588,NaN,2.0,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.000000,48.000000,NaN,2.0,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,51.000000,51.000000,NaN,2.0,NaN,LIVING COMEDOR CON PISOS DE PO...
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.000000,67.000000,NaN,2.0,2600,EXCELENTE 2 AMB- APTO PROFESIO...
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,47.000000,47.000000,NaN,2.0,NaN,IMPECABLE MONOAMBIENTE- DIVISI...


In [9]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                       1368
rooms                      14166
expenses                    2543
description                14166
dtype: int64

<br><B><FONT FACE="Arial" SIZE="5">Preparo los datos</FONT></B><br/>

In [26]:
#FILTRO LOS DATOS ANOMALOS

data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'rooms', 'lat', 'lon']].dropna()


In [27]:

#filtro
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)) & \
               ((data_float.rooms >= 0.0) & (data_float.rooms <= 200.0)), :]


data_float.shape
#(68586, 6)

(68586, 6)

## Uso PCA como reductor de dimensiones para sumar dos nuevas dimensiones al dataset

In [39]:
data_float_pca = data_float.loc[:,['surface_total_in_m2', 'surface_covered_in_m2',\
                          'rooms', 'lat', 'lon']]
data_float_pca

,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon
0,500.0,500.0,6.0,-34.643701,-58.651548
1,135.0,135.0,4.0,-34.546433,-58.545713
2,330.0,310.0,6.0,-34.345731,-58.855461
3,0.0,218.0,4.0,-34.735512,-58.398874
4,30.0,30.0,2.0,-34.649843,-58.657190
5,47.0,47.0,2.0,-34.651906,-58.644602
6,294.0,100.0,4.0,-34.767167,-58.215465
7,79.0,70.0,3.0,-34.578408,-58.495869
8,75.0,65.0,2.0,-34.401696,-58.640455
9,0.0,41.0,1.0,-34.588197,-58.501429


In [42]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(data_float_pca)
red = pca.transform(data_float_pca)
print(pca.singular_values_)

[ 221124.11604037   56617.79278621]


In [43]:
red

array([[ 360.49208984,  396.48195478],
       [ -12.29734775,   39.43788857],
       [ 186.4328821 ,  210.19386952],
       ..., 
       [-122.60252013,  -66.21242102],
       [   1.57033499,   33.13912056],
       [ 242.64348777,   33.9369007 ]])

In [44]:
d = {'col1': red[:,0], 'col2': red[:,1]}
red_data = pd.DataFrame(data=d)

data_float.reset_index(inplace=True)
data_float['PCA1'] = red_data['col1']
data_float['PCA2'] = red_data['col2']
data_float.tail()

,level_0,index,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon,PCA1,PCA2
68581,68581,295459,50000.0,70.0,200.0,3.0,-38.030114,-57.574442,-75.880045,105.823087
68582,68582,295460,74000.0,39.0,39.0,2.0,-38.002849,-57.544908,-110.346216,-54.472262
68583,68583,295461,45000.0,27.0,27.0,1.0,-37.996638,-57.550819,-122.602520,-66.212421
68584,68584,295462,349000.0,149.0,129.0,5.0,-37.099113,-56.840540,1.570335,33.139121
68585,68585,295463,75000.0,390.0,135.0,4.0,-37.837561,-58.259803,242.643488,33.936901


In [45]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                       1368
rooms                      14166
expenses                    2543
description                14166
dtype: int64

In [46]:
test_float_pca = test_float.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                    'rooms', 'lat', 'lon']]
test_float_pca

,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon
0,0.0,359.471588,2.0,-34.610988,-58.363464
1,0.0,359.471588,2.0,-34.629923,-58.465820
2,53.0,48.000000,2.0,-34.589363,-58.412880
3,39.0,39.000000,2.0,-34.629923,-58.465820
4,51.0,51.000000,2.0,-34.629923,-58.465820
5,39.0,39.000000,2.0,-34.629923,-58.465820
6,39.0,39.000000,2.0,-34.629923,-58.465820
7,39.0,39.000000,2.0,-34.629923,-58.465820
8,67.0,67.000000,2.0,-34.578721,-58.457095
9,47.0,47.000000,2.0,-34.629923,-58.465820


In [47]:
red_test = pca.transform(test_float_pca)
red_test

array([[-142.42398881,  266.76548379],
       [-142.42398809,  266.76548826],
       [ -96.1555152 ,  -45.77644375],
       ..., 
       [  71.68265842,   37.62208294],
       [ -87.15761146,  -45.97059761],
       [ 219.54080393,   29.43417778]])

In [48]:
d = {'col1': red_test[:,0], 'col2': red_test[:,1]}
red_test_data = pd.DataFrame(data=d)

#test_float.reset_index(inplace=True)
test_float['PCA1'] = red_test_data['col1']
test_float['PCA2'] = red_test_data['col2']
test_float.tail()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,PCA1,PCA2
14161,2245938,2017-08-01,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6115752,-58.3630313",-34.611575,-58.363031,109.0,96.0,NaN,2.0,8500,"Edificio.""Terrazas del Dique"".Al Frente.Balcón...",-39.133072,1.004226
14162,2245978,2017-08-17,ph,venta,Temperley,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,"-34.7821307,-58.4157403",-34.782131,-58.415740,38.0,26.0,NaN,2.0,NaN,Venta de imperdible PH con patio 2 AMBIENTE...,-111.626606,-67.447690
14163,2246083,2017-08-01,casa,venta,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,"-34.7585252,-58.4097202",-34.758525,-58.409720,219.0,135.0,NaN,2.0,NaN,"Venta de Casa 4 AMBIENTES en Lomas de Zamora, ...",71.682658,37.622083
14164,2246089,2017-08-25,departamento,venta,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"-34.563221078,-58.4610526805",-34.563221,-58.461053,62.0,48.0,NaN,2.0,4128,DEPARTAMENTO EN VENTA / ALQUILER | BELGRANO ...,-87.157611,-45.970598
14165,2246127,2017-08-12,casa,venta,Ciudad Evita,|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza...,Argentina,Bs.As. G.B.A. Zona Oeste,"-34.7091916902,-58.5302893817",-34.709192,-58.530289,367.0,130.0,NaN,4.0,NaN,Venta de Chalet 4 AMBIENTES en CIRCUNSCRIPCIÓN...,219.540804,29.434178


<br><B><FONT FACE="Arial" SIZE="5">Predicción usando XGB</FONT></B><br/>

In [53]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'rooms', 'PCA1', 'PCA2']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [54]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls', 'criterion': 'mse'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [55]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.838/0.717
#Precisión Gradient Boosting train/test  0.686/0.325 cuando la precisión era mejor, nuestras predicciones eran 
#peores. Habría que ver cómo mide la precisión ésta cosa

Precisión Gradient Boosting train/test  0.720/0.324


In [56]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms', 'PCA1', 'PCA2']])
test['price_aprox_usd_XGB'].head(100)

0     462710.462828
1     462710.462828
2      93127.951932
3      78135.736865
4     110383.583554
5      78135.736865
6      78135.736865
7      78135.736865
8     116384.416076
9      93127.951932
10     93127.951932
11     93127.951932
12    116384.416076
13    227772.812422
14     78135.736865
15     78135.736865
16     78135.736865
17     78135.736865
18     78135.736865
19     78135.736865
20     78135.736865
21     78135.736865
22    110383.583554
23    110383.583554
24    110383.583554
25    110383.583554
26    110383.583554
27     78135.736865
28     78135.736865
29     78135.736865
          ...      
70    116384.416076
71     96432.766677
72    167214.810031
73    167214.810031
74    116384.416076
75    116384.416076
76    116384.416076
77    116384.416076
78    184809.902605
79    116384.416076
80    156401.878157
81    184809.902605
82    199337.873131
83    153016.308900
84    153016.308900
85     93127.951932
86    183537.676882
87     78135.736865
88    158677.125962


In [96]:
#Genero el csv con los nombres de columnas requeridos por Kaggle
#test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
#test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingBuenosAires.csv', index=False)

<br><B><FONT FACE="Arial" SIZE="5">XGBoosting con lat y lon</FONT></B><br/>

In [57]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'rooms', 'lat', 'lon', 'PCA1', 'PCA2']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [58]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [59]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.905/0.736
#Precisión Gradient Boosting train/test  0.864/0.549


#Precisión Gradient Boosting train/test  1.000/0.963

Precisión Gradient Boosting train/test  0.864/0.559


In [60]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms',  'lat', 'lon', 'PCA1', 'PCA2']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingLatLonGenerico.csv', index=False)
test_float_reduced['price_usd'].head(100)

0     939816.960103
1     525552.952550
2     112420.900948
3      77641.577987
4     105794.226261
5      77641.577987
6      77641.577987
7      77641.577987
8     129958.839679
9      87775.033913
10     87775.033913
11     87775.033913
12    109531.704515
13    509004.952137
14     77641.577987
15     77641.577987
16     77641.577987
17     77641.577987
18     77641.577987
19     77641.577987
20     77641.577987
21     77641.577987
22    105425.937740
23    105794.226261
24    105794.226261
25    105794.226261
26    105794.226261
27     77641.577987
28     77641.577987
29     95476.443057
          ...      
70    109531.704515
71     81052.756217
72    156604.481770
73    156604.481770
74    109531.704515
75    109531.704515
76    109531.704515
77    109531.704515
78    314312.145894
79    109531.704515
80    130704.274076
81    314312.145894
82    161649.323170
83    115488.683454
84    148473.742362
85     87775.033913
86    255227.561789
87    102114.268890
88    130704.274076


<br><B><FONT FACE="Arial" SIZE="5">Uso grid-search para estimar los parametros</FONT></B><br/>

In [61]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'rooms',  'lat', 'lon', 'PCA1', 'PCA2']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [62]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_test1 = {'n_estimators':list(range(1,600,10))}

gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500,\
min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1 ,max_features='sqrt',\
#subsample=0.8, random_state=10), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
                        
gsearch1.fit(X_train, y_train)
#Me fijo el resultado del parametro

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_samples_split=500,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=10, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

In [63]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


([mean: -68321601018.76437, std: 40974986427.99329, params: {'n_estimators': 1},
  mean: -53371023567.64619, std: 38027355358.61302, params: {'n_estimators': 11},
  mean: -47899685961.28407, std: 36001117223.91906, params: {'n_estimators': 21},
  mean: -45988628753.64748, std: 33906836973.33976, params: {'n_estimators': 31},
  mean: -44711274073.20656, std: 32246035996.64996, params: {'n_estimators': 41},
  mean: -43800396560.56451, std: 30489813968.46945, params: {'n_estimators': 51},
  mean: -43117332124.24018, std: 30224488803.04722, params: {'n_estimators': 61},
  mean: -42374424218.43252, std: 29482386945.40467, params: {'n_estimators': 71},
  mean: -41974918711.02219, std: 28791867703.81279, params: {'n_estimators': 81},
  mean: -41886141592.53328, std: 28345199624.63908, params: {'n_estimators': 91},
  mean: -41775107769.26991, std: 28165897836.38205, params: {'n_estimators': 101},
  mean: -41616045263.90181, std: 27931366590.87670, params: {'n_estimators': 111},
  mean: -413946

In [64]:
#Encontrar los parametros max_depth y min_samples_split
#Aca ya uso n_estimators=491
param_test2 = {'max_depth':list(range(2,16,2)), 'min_samples_split':list(range(200,1001,10))}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=421, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -49506934023.55379, std: 27750147833.30687, params: {'min_samples_split': 200, 'max_depth': 2},
  mean: -48849338484.51121, std: 27906271339.47131, params: {'min_samples_split': 210, 'max_depth': 2},
  mean: -48941891847.02287, std: 28039416684.02363, params: {'min_samples_split': 220, 'max_depth': 2},
  mean: -49046144771.36509, std: 28304047374.98861, params: {'min_samples_split': 230, 'max_depth': 2},
  mean: -48873435166.39520, std: 27866764134.54739, params: {'min_samples_split': 240, 'max_depth': 2},
  mean: -48430110001.34969, std: 28368536502.23388, params: {'min_samples_split': 250, 'max_depth': 2},
  mean: -48942523658.17120, std: 28084483299.99674, params: {'min_samples_split': 260, 'max_depth': 2},
  mean: -48394015270.27795, std: 28257579029.97832, params: {'min_samples_split': 270, 'max_depth': 2},
  mean: -48480911882.76067, std: 28223079442.02661, params: {'min_samples_split': 280, 'max_depth': 2},
  mean: -48986531237.39915, std: 27605740689.78560, params: {'mi

In [65]:
#Encontrar el parametro min_samples_leaf
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210

param_test3 = {'min_samples_leaf':list(range(10,100,1))}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=421, min_samples_split=260, max_depth=8,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train, y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -41579239601.24598, std: 21371990285.43526, params: {'min_samples_leaf': 10},
  mean: -40903473678.35507, std: 20313459855.26038, params: {'min_samples_leaf': 11},
  mean: -41663625417.57839, std: 20076287839.91192, params: {'min_samples_leaf': 12},
  mean: -42057236471.65730, std: 21825716974.29763, params: {'min_samples_leaf': 13},
  mean: -41901220155.81968, std: 20694320370.21594, params: {'min_samples_leaf': 14},
  mean: -41890839013.61428, std: 20644806797.78542, params: {'min_samples_leaf': 15},
  mean: -40959659322.07124, std: 21366167499.92435, params: {'min_samples_leaf': 16},
  mean: -41492857381.60858, std: 21243134019.46553, params: {'min_samples_leaf': 17},
  mean: -40910831148.42123, std: 20725135846.09555, params: {'min_samples_leaf': 18},
  mean: -41403239114.18275, std: 21610067753.82707, params: {'min_samples_leaf': 19},
  mean: -40931120655.88329, std: 21992542075.23531, params: {'min_samples_leaf': 20},
  mean: -40765377516.52920, std: 22017717460.43879, pa

In [66]:
#Encontrar el parametro max_features
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf': 65

param_test4 = {'max_features':list(range(1,6,1))}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=421, max_depth=8, min_samples_split=260, min_samples_leaf=83
                                                              , subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train, y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: -39096187334.76337, std: 23220069779.12663, params: {'max_features': 1},
  mean: -37800826722.25500, std: 21666058344.57601, params: {'max_features': 2},
  mean: -38671571632.14174, std: 22550727891.91913, params: {'max_features': 3},
  mean: -38807389040.01105, std: 22319126957.67247, params: {'max_features': 4},
  mean: -38048798480.83896, std: 21893048169.02112, params: {'max_features': 5}],
 {'max_features': 2},
 -37800826722.255005)

In [67]:
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf': 65, 'max_features': 4
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 421, 'max_depth': 8, 'min_samples_split': 260, 'min_samples_leaf':83 , 'max_features': 2,
          'learning_rate': 0.1}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.823/0.767
#Precisión Gradient Boosting train/test  0.875/0.526

Precisión Gradient Boosting train/test  0.801/0.523


In [68]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms',  'lat', 'lon', 'PCA1', 'PCA2']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS.csv', index=False)

## Uso grid-search para estimar los parametros, ésta vez todos juntos


In [ ]:
#Creo X e Y
#feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',  'lat', 'lon']
#TRAIN = data_float[:2600]
#TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
#X_train = TRAIN[feature_cols]
#X_test = TEST[feature_cols]

#y_train = TRAIN.price_aprox_usd
#y_test = TEST.price_aprox_usd

In [ ]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
#from sklearn.grid_search import GridSearchCV
#from sklearn.ensemble import GradientBoostingRegressor

#param_test1 = {'n_estimators':list(range(1,500,5)), 'max_depth':list(range(2,16,2)),\
#               'min_samples_split':list(range(200,1000,50)), 'min_samples_leaf':list(range(10,100,5)),\
#                'max_features':list(range(1,4,1))}

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, subsample=0.8, random_state=10),\
                param_grid = param_test1, scoring='neg_mean_squared_error', n_jobs=4,iid=False, cv=5)
                        
#gsearch1.fit(X_train, y_train)

In [ ]:
# {'max_depth': 10,
#  'max_features': 3,
#  'min_samples_leaf': 40,
#  'min_samples_split': 200,
#  'n_estimators': 261},

#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

### Con estos hiper-parametros estimados, hago una prediccion

In [ ]:
# 'n_estimators':261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
#from sklearn import ensemble
#params = {'n_estimators': 261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3,
#          'learning_rate': 0.1}


#clf = ensemble.GradientBoostingRegressor(**params)

#clf.fit(X_train, y_train)

#Me fijo la precision
#print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
#      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.857/0.837

In [ ]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS2.csv', index=False)